In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


# import data

In [2]:
data = pd.read_parquet("outputs_new/dedupe_data_out_version_links.parquet") #dedupe_data_out

data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084855,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None
8084856,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None
8084857,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None
8084858,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,parent,None,None,chen_chi_yi,investigation of the formal

In [3]:
import pandas as pd
import numpy as np

# Columns to duplicate
cols_to_duplicate = ['records_hierarchy', 'parent_record_id','dup_group_id']

# Duplicate the columns with new names
data[['records_hierarchy_unify_version_backup', 'parent_record_id_unify_version_backup', 'dup_group_id_unify_version_backup']] = data[cols_to_duplicate].copy()

data.columns


Index(['record_id', 'server_name', 'backend', 'doi', 'doi_url',
       'landing_page_url', 'title', 'authors_flat', 'institutions_flat',
       'countries_flat', 'relations_json', 'version_label', 'is_version_of',
       'is_preprint_of', 'has_preprint', 'has_review', 'has_published_version',
       'published_version_ids_json', 'version_of_ids_json', 'update_to_json',
       'raw_relationships_json', 'records_hierarchy', 'date_first_seen',
       'publication_year_first_seen', 'records_hierarchy_backup',
       'parent_record_id', 'dup_group_id', 'authors_fp_tokenbag',
       'title_clean_v2', 'authors_fp_last_initial', 'authors_fp_last',
       'records_hierarchy_dedupe_backup', 'parent_record_id_dedupe_backup',
       'dup_group_id_dedupe_backup', 'records_hierarchy_unify_version_backup',
       'parent_record_id_unify_version_backup',
       'dup_group_id_unify_version_backup'],
      dtype='object')

In [4]:
# any cluster id should correspond to one connected component
print(data["records_hierarchy"].value_counts())


records_hierarchy
parent                                            7458929
parent - duplicate (SSRN)                          117973
parent - duplicate (Zenodo)                         92311
parent - duplicate (version-link)                   83495
parent - duplicate (arXiv)                          54833
                                                   ...   
parent - duplicate (WikiJournal of Humanities)          2
parent - duplicate (Encyclopedia)                       1
parent - duplicate (BioHackrXiv)                        1
parent - duplicate (Therapoid)                          1
parent - duplicate (searchRxiv)                         1
Name: count, Length: 106, dtype: int64


# unify/merge group ids across stages

In [5]:
# import pandas as pd

# class UnionFind:
#     def __init__(self):
#         self.parent = {}

#     def find(self, x):
#         x = str(x)
#         if x not in self.parent:
#             self.parent[x] = x
#         # path compression
#         while self.parent[x] != x:
#             self.parent[x] = self.parent[self.parent[x]]
#             x = self.parent[x]
#         return x

#     def union(self, a, b):
#         ra, rb = self.find(a), self.find(b)
#         if ra != rb:
#             # deterministic: attach larger root to smaller root (stable)
#             if ra < rb:
#                 self.parent[rb] = ra
#             else:
#                 self.parent[ra] = rb

# def merge_groups_from_parent_links(
#     df: pd.DataFrame,
#     *,
#     record_id_col="record_id",
#     parent_id_col="parent_record_id",
#     hierarchy_col="records_hierarchy",
#     group_id_col="dup_group_id",
#     only_when_duplicate_prefix="parent - duplicate",
# ) -> pd.DataFrame:
#     """
#     Merge cluster IDs so that any duplicate link (child -> parent)
#     forces both records to share a single canonical dup_group_id.

#     Works even if different stages created different group ids.
#     """
#     # rows that are duplicates (children)
#     h = df[hierarchy_col].astype("string").fillna("")
#     dup_mask = h.str.startswith(only_when_duplicate_prefix)

#     # edges child -> parent
#     edges = df.loc[dup_mask, [record_id_col, parent_id_col]].dropna()
#     if edges.empty:
#         return df

#     uf = UnionFind()

#     # Union by record IDs (most robust)
#     for child, parent in edges.itertuples(index=False):
#         uf.union(child, parent)

#     # Map each record_id to component root
#     roots = df[record_id_col].astype("string").map(uf.find)

#     # Canonical group id = component root (or hash of it if you prefer)
#     # This ensures *all stages collapse into one group id*.
#     df[group_id_col] = roots

#     # Optional: ensure parents have parent_id = NA (clean)
#     parent_mask = h.str.lower().str.strip().eq("parent")
#     if parent_id_col in df.columns:
#         df.loc[parent_mask, parent_id_col] = pd.NA

#     return df


In [6]:
# # after stage i
# data_group_merge = merge_groups_from_parent_links(data_clean,
#     hierarchy_col="records_hierarchy_copy2",)

# data_group_merge.to_pickle("outputs/dedupe_data_group_merge.pkl")

In [7]:
import pandas as pd

class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        x = str(x)
        if x not in self.parent:
            self.parent[x] = x
        # path compression
        while self.parent[x] != x:
            self.parent[x] = self.parent[self.parent[x]]
            x = self.parent[x]
        return x

    def union(self, a, b):
        ra, rb = self.find(a), self.find(b)
        if ra != rb:
            # deterministic attach (not “parent aware” — parent selection happens later)
            if ra < rb:
                self.parent[rb] = ra
            else:
                self.parent[ra] = rb


def unify_groups_by_parent_links_with_parent_selection(
    df: pd.DataFrame,
    *,
    record_id_col="record_id",
    server_col="server_name",
    hierarchy_col="records_hierarchy",
    parent_id_col="parent_record_id",
    group_id_col="dup_group_id",
    date_col="date_first_seen",   # can be None
    choose_parent="oldest",       # "oldest" | "newest"
    only_when_duplicate_prefix="parent - duplicate",
    overwrite_mode="duplicate_only",  # "duplicate_only" | "parents_and_duplicates" | "any"
    set_hierarchy=True,
    add_members_list_col=False,
    members_list_col="version_record_ids",  # optional list col
) -> tuple[pd.DataFrame, dict]:
    """
    Merge/Unify duplicate clusters across stages using existing duplicate links (child -> parent),
    then SELECT a single canonical parent per connected component and set:
      - dup_group_id = canonical_parent_record_id
      - parent_record_id = canonical_parent_record_id for all non-parent members
      - records_hierarchy updated accordingly (optional)

    This solves the “different stages produced different group ids” problem AND guarantees that
    dup_group_id is the parent record_id you chose by policy.
    """

    metrics = {
        "n_rows": int(len(df)),
        "n_dupe_edges": 0,
        "n_involved_records": 0,
        "n_components": 0,
        "n_components_size_ge2": 0,
        "n_rows_updated_group_id": 0,
        "n_rows_updated_parent_id": 0,
        "n_rows_updated_hierarchy": 0,
        "n_parents_set": 0,
        "n_duplicates_set": 0,
        "choose_parent": choose_parent,
        "date_col_used": date_col if (date_col in df.columns) else None,
        "overwrite_mode": overwrite_mode,
    }

    if record_id_col not in df.columns:
        raise ValueError(f"Missing column: {record_id_col}")
    if parent_id_col not in df.columns:
        raise ValueError(f"Missing column: {parent_id_col}")
    if hierarchy_col not in df.columns:
        raise ValueError(f"Missing column: {hierarchy_col}")
    if server_col not in df.columns:
        # not strictly required, but helps set “duplicate (server)” suffix
        df[server_col] = pd.NA

    # -------- 1) build edges from duplicate rows only --------
    h = df[hierarchy_col].astype("string").fillna("")
    dup_mask = h.str.startswith(only_when_duplicate_prefix)

    edges = df.loc[dup_mask, [record_id_col, parent_id_col]].dropna()
    metrics["n_dupe_edges"] = int(len(edges))
    if edges.empty:
        return df, metrics

    # -------- 2) Union-Find on involved records only --------
    uf = UnionFind()
    # union edges child <-> parent (component)
    for child, parent in edges.itertuples(index=False):
        uf.union(child, parent)

    involved = pd.Index(pd.unique(edges[[record_id_col, parent_id_col]].values.ravel("K"))).astype(str)
    metrics["n_involved_records"] = int(len(involved))

    # Map record_id -> component root (only for involved IDs)
    root_map = {rid: uf.find(rid) for rid in involved}

    # -------- 3) Build a compact “involved” table for parent selection --------
    # Only rows whose record_id is involved
    rid_series = df[record_id_col].astype("string")
    involved_mask = rid_series.isin(involved)
    inv = df.loc[involved_mask, [record_id_col, server_col, hierarchy_col, parent_id_col]].copy()

    inv["_root"] = inv[record_id_col].astype("string").map(root_map)

    # date + tie-breakers (small table compared to 8M)
    if date_col and date_col in df.columns:
        inv["_dt"] = pd.to_datetime(df.loc[involved_mask, date_col], errors="coerce")
    else:
        inv["_dt"] = pd.NaT

    # numeric tie-breaker: extract digits from record_id (cheap, stable)
    inv["_rid_num"] = (
        inv[record_id_col].astype("string").str.extract(r"(\d+)")[0]
    )
    inv["_rid_num"] = pd.to_numeric(inv["_rid_num"], errors="coerce")

    # -------- 4) Choose canonical parent per component --------
    # Prefer records currently labeled "parent" if present in the component.
    inv["_is_parent_label"] = inv[hierarchy_col].astype("string").str.lower().str.strip().eq("parent")

    # Sort policy:
    # - primary: prefer parent-labeled rows (descending True first)
    # - then date (oldest/newest)
    # - then rid_num
    # - then record_id (final stable tie-break)
    asc_dt = True if choose_parent == "oldest" else False
    asc_num = True if choose_parent == "oldest" else False
    asc_id = True

    inv_sorted = inv.sort_values(
        by=["_root", "_is_parent_label", "_dt", "_rid_num", record_id_col],
        ascending=[True, False, asc_dt, asc_num, asc_id],
        na_position="last",
        kind="mergesort",  # stable
    )

    # canonical parent = first row per root
    canon = inv_sorted.groupby("_root", sort=False).head(1)[["_root", record_id_col]]
    canon = canon.rename(columns={record_id_col: "_canon_parent_rid"})
    canon_map = canon.set_index("_root")["_canon_parent_rid"].to_dict()

    # component sizes (for metrics + optional members list)
    comp_sizes = inv["_root"].value_counts()
    metrics["n_components"] = int(comp_sizes.shape[0])
    metrics["n_components_size_ge2"] = int((comp_sizes >= 2).sum())

    # -------- 5) Decide which rows are allowed to be overwritten --------
    # We will update *involved rows* only, but with safety modes.
    if overwrite_mode == "duplicate_only":
        allow_mask = involved_mask & dup_mask
    elif overwrite_mode == "parents_and_duplicates":
        allow_mask = involved_mask & (dup_mask | h.str.lower().str.strip().eq("parent"))
    elif overwrite_mode == "any":
        allow_mask = involved_mask
    else:
        raise ValueError("overwrite_mode must be: duplicate_only | parents_and_duplicates | any")

    # For any involved row (even if not overwritten), we still need to compute its canonical parent
    canon_parent_for_row = rid_series.map(root_map).map(canon_map)

    # -------- 6) Update dup_group_id to canonical parent id --------
    if group_id_col not in df.columns:
        df[group_id_col] = pd.NA

    before_gid = df.loc[allow_mask, group_id_col].astype("string")
    df.loc[allow_mask, group_id_col] = canon_parent_for_row.loc[allow_mask].values
    after_gid = df.loc[allow_mask, group_id_col].astype("string")
    metrics["n_rows_updated_group_id"] = int((before_gid != after_gid).sum())

    # -------- 7) Update parent_record_id for non-parent members --------
    # Determine which record is the canonical parent within each component
    is_canon_parent = rid_series.eq(canon_parent_for_row)

    # For rows we are allowed to update:
    # - canonical parent: parent_record_id = NA
    # - others: parent_record_id = canonical parent rid
    upd_parent_mask = allow_mask.copy()
    if parent_id_col not in df.columns:
        df[parent_id_col] = pd.NA

    before_pid = df.loc[upd_parent_mask, parent_id_col].astype("string")

    df.loc[upd_parent_mask & is_canon_parent, parent_id_col] = pd.NA
    df.loc[upd_parent_mask & (~is_canon_parent), parent_id_col] = canon_parent_for_row.loc[upd_parent_mask & (~is_canon_parent)].values

    after_pid = df.loc[upd_parent_mask, parent_id_col].astype("string")
    metrics["n_rows_updated_parent_id"] = int((before_pid != after_pid).sum())

    # -------- 8) Update hierarchy labels (optional) --------
    if set_hierarchy:
        before_h = df.loc[allow_mask, hierarchy_col].astype("string")

        # parents
        df.loc[allow_mask & is_canon_parent, hierarchy_col] = "parent"
        metrics["n_parents_set"] = int((allow_mask & is_canon_parent).sum())

        # duplicates: keep server suffix, use their own server_name
        dup_rows = allow_mask & (~is_canon_parent)
        metrics["n_duplicates_set"] = int(dup_rows.sum())

        # build suffix "(server)" if available
        srv = df.loc[dup_rows, server_col].astype("string").fillna("")
        suffix = srv.where(srv != "", "unknown")
        df.loc[dup_rows, hierarchy_col] = "parent - duplicate (" + suffix + ")"

        after_h = df.loc[allow_mask, hierarchy_col].astype("string")
        metrics["n_rows_updated_hierarchy"] = int((before_h != after_h).sum())

    # -------- 9) Optional: add members list per canonical parent --------
    # Warning: this column can be large; for 8M rows you may prefer exporting a separate table.
    if add_members_list_col:
        # Only build members for components size>=2 (involved components)
        inv2 = pd.DataFrame({
            "_canon_parent": canon_parent_for_row.loc[involved_mask].values,
            record_id_col: df.loc[involved_mask, record_id_col].astype("string").values
        })
        members = inv2.groupby("_canon_parent")[record_id_col].apply(lambda s: ";".join(sorted(set(s)))).reset_index()
        members = members.rename(columns={"_canon_parent": group_id_col, record_id_col: members_list_col})

        # merge back on dup_group_id (which is canonical parent id)
        df = df.merge(members, on=group_id_col, how="left")

    return df, metrics


In [8]:
data_group_merge, metrics = unify_groups_by_parent_links_with_parent_selection(
    data,
    hierarchy_col="records_hierarchy",
    overwrite_mode="any",  # "duplicate_only" | "parents_and_duplicates" | "any"
)
data_group_merge

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084855,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None,parent,None,None
8084856,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None,parent,None,None
8084857,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None,parent,None,None
8084858,openalex::W999790414,viXra,open

In [9]:
import json

with open("outputs_new/dedupe_group_merge_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

data_group_merge.to_pickle("outputs_new/dedupe_data_group_merge.pkl")

In [10]:
print(metrics)

data_group_merge["dup_group_id"].nunique()
data_group_merge["records_hierarchy"].value_counts()#.head()


{'n_rows': 8084860, 'n_dupe_edges': 625931, 'n_involved_records': 1119263, 'n_components': 493332, 'n_components_size_ge2': 485490, 'n_rows_updated_group_id': 866545, 'n_rows_updated_parent_id': 17590, 'n_rows_updated_hierarchy': 283094, 'n_parents_set': 493332, 'n_duplicates_set': 617740, 'choose_parent': 'oldest', 'date_col_used': 'date_first_seen', 'overwrite_mode': 'any'}


records_hierarchy
parent                                            7467120
parent - duplicate (SSRN)                          135606
parent - duplicate (Zenodo)                         98767
parent - duplicate (arXiv)                          85008
parent - duplicate (Research Square)                59931
                                                   ...   
parent - duplicate (SAE Mobilus®)                       1
parent - duplicate (Therapoid)                          1
parent - duplicate (WikiJournal of Humanities)          1
parent - duplicate (WikiJournal of Science)             1
parent - duplicate (searchRxiv)                         1
Name: count, Length: 109, dtype: int64

In [11]:
print(metrics)

data_group_merge["dup_group_id"].nunique()
data_group_merge["records_hierarchy"].value_counts().head()


{'n_rows': 8084860, 'n_dupe_edges': 625931, 'n_involved_records': 1119263, 'n_components': 493332, 'n_components_size_ge2': 485490, 'n_rows_updated_group_id': 866545, 'n_rows_updated_parent_id': 17590, 'n_rows_updated_hierarchy': 283094, 'n_parents_set': 493332, 'n_duplicates_set': 617740, 'choose_parent': 'oldest', 'date_col_used': 'date_first_seen', 'overwrite_mode': 'any'}


records_hierarchy
parent                                  7467120
parent - duplicate (SSRN)                135606
parent - duplicate (Zenodo)               98767
parent - duplicate (arXiv)                85008
parent - duplicate (Research Square)      59931
Name: count, dtype: int64

In [12]:
clustered = data_group_merge[data_group_merge["dup_group_id"].notna()].copy()

n_clusters = clustered["dup_group_id"].nunique()
sizes = clustered.groupby("dup_group_id")["record_id"].size()

print("clusters:", n_clusters)
print("min/median/p95/max:", sizes.min(), sizes.median(), sizes.quantile(0.95), sizes.max())


clusters: 495664
min/median/p95/max: 1 2.0 3.0 702


In [13]:
sizes = data_group_merge.dropna(subset=["dup_group_id"]).groupby("dup_group_id")["record_id"].size()
print(sizes.describe())

count    495664.000000
mean          2.246288
std           2.277007
min           1.000000
25%           2.000000
50%           2.000000
75%           2.000000
max         702.000000
Name: record_id, dtype: float64


In [14]:
top = sizes.sort_values(ascending=False).head(20)
print(top)


dup_group_id
datacite::10.5281/zenodo.14289192       702
datacite::10.5281/zenodo.15609432       569
datacite::10.22004/ag.econ.138684       450
datacite::10.22004/ag.econ.136112       390
datacite::10.22004/ag.econ.286599       301
datacite::10.5281/zenodo.15832876       286
datacite::10.5281/zenodo.15161515       261
crossref::10.31219/osf.io/ed2a9         210
crossref::10.32388/174914               184
datacite::10.5281/zenodo.6784019        182
datacite::10.5281/zenodo.14599183       177
datacite::10.5281/zenodo.15631517       167
crossref::10.2139/ssrn.2721313          164
datacite::10.5281/zenodo.15408957       162
datacite::10.5281/zenodo.15690627       159
crossref::10.31237/osf.io/7j3am         150
crossref::10.21055/preprints-3112061    136
datacite::10.5281/zenodo.1297654        135
datacite::10.5281/zenodo.17172763       128
datacite::10.5281/zenodo.16684574       119
Name: record_id, dtype: int64


In [15]:

data_group_merge.head(3)


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None,parent,None,None


In [16]:
pattern = "Search for a heavy resonance decaying into"


mask = data_group_merge['title'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

6


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
335713,openalex::W2921909056,DSpace@MIT,openalex,None,None,http://hdl.handle.net/1721.1/117537,Search for a heavy resonance decaying into a Z...,Daniel Robert Abercrombie; Brandon Leigh Allen...,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2018-07-01,2018,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
4739525,datacite::10.48550/arxiv.2111.10216,arXiv,datacite,10.48550/arxiv.2111.10216,https://doi.org/10.48550/arxiv.2111.10216,https://arxiv.org/abs/2111.10216,Search for a heavy resonance decaying into a t...,CMS Collaboration,None,None,"[{""relatedIdentifier"": ""10.1007/jhep04(2022)04...",2,,10.1007/jhep04(2022)048,,,true,None,None,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-02-20,2022,parent,None,None,cms_collaboration,search for a heavy resonance decaying into a t...,collaboration|c,None,parent,None,None,parent,None,None
5975676,datacite::10.48550/arxiv.1803.03838,arXiv,datacite,10.48550/arxiv.1803.03838,https://doi.org/10.48550/arxiv.1803.03838,https://arxiv.org/abs/1803.03838,Search for a heavy resonance decaying into a Z...,CMS Collaboration,None,None,"[{""relatedIdentifier"": ""10.1007/jhep07(2018)07...",2,,10.1007/jhep07(2018)075,,,true,None,None,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-03,2022,parent,None,None,cms_collaboration,search for a heavy resonance decaying into a z...,collaboration|c,None,parent,None,None,parent,None,None
5977724,datacite::10.48550/arxiv.1803.10093,arXiv,datacite,10.48550/arxiv.1803.10093,https://doi.org/10.48550/arxiv.1803.10093,https://arxiv.org/abs/1803.10093,Search for a heavy resonance decaying into a Z...,CMS Collaboration,None,None,"[{""relatedIdentifier"": ""10.1007/jhep09(2018)10...",2,,10.1007/jhep09(2018)101,,,true,None,None,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-03,2022,parent,None,None,cms_collaboration,search for a heavy resonance decaying into a z...,collaboration|c,None,parent,None,None,parent,None,None
6550333,datacite::10.48550/arxiv.hep-ex/0606018,arXiv,datacite,10.48550/arxiv.hep-ex/0606018,https://doi.org/10.48550/arxiv.hep-ex/0606018,https://arxiv.org/abs/hep-ex/0606018,Search for a heavy resonance decaying into a Z...,"D0 Collaboration; Abazov, V. M.",None,None,"[{""relatedIdentifier"": ""10.1103/physrevd.74.01...",1,,10.1103/physrevd.74.011104,,,true,None,None,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2022-03-17,2022,parent,None,None,abazov_m_v;collaboration_d0,search for a heavy resonance decaying into a z...,abazov|v;collaboration|d,None,parent,None,None,parent,None,None
7628808,datacite::10.48550/arxiv.2411.00202,arXiv,datacite,10.48550/arxiv.2411.00202,https://doi.org/10.48550/arxiv.2411.00202,https://arxiv.org/abs/2411.00202,Search for a heavy resonance decaying into a Z...,CMS Collaboration,None,None,"[{""relatedIdentifier"": ""10.1007/jhep02(2025)08...",2,,10.1007/jhep02(2025)089,,,true,None,None,None,"{""client"": {""data"": {""id"": ""arxiv.content"", ""t...",parent,2024-11-04,2024,parent,None,None,cms_collaboration,search for a heavy resonance decaying into a z...,collaboration|c,None,parent,None,None,parent,None,None


In [17]:
data_group_merge.columns.tolist()

['record_id',
 'server_name',
 'backend',
 'doi',
 'doi_url',
 'landing_page_url',
 'title',
 'authors_flat',
 'institutions_flat',
 'countries_flat',
 'relations_json',
 'version_label',
 'is_version_of',
 'is_preprint_of',
 'has_preprint',
 'has_review',
 'has_published_version',
 'published_version_ids_json',
 'version_of_ids_json',
 'update_to_json',
 'raw_relationships_json',
 'records_hierarchy',
 'date_first_seen',
 'publication_year_first_seen',
 'records_hierarchy_backup',
 'parent_record_id',
 'dup_group_id',
 'authors_fp_tokenbag',
 'title_clean_v2',
 'authors_fp_last_initial',
 'authors_fp_last',
 'records_hierarchy_dedupe_backup',
 'parent_record_id_dedupe_backup',
 'dup_group_id_dedupe_backup',
 'records_hierarchy_unify_version_backup',
 'parent_record_id_unify_version_backup',
 'dup_group_id_unify_version_backup']

# creates inspection packets + an HTML report (clickable) + the key CSVs, all in outputs/

In [18]:
from __future__ import annotations

from pathlib import Path
import json
import random
import pandas as pd
import re
import hashlib
def _gid_slug(gid: str) -> str:
    safe = _safe_filename(gid, 60)
    h = hashlib.md5(str(gid).encode("utf-8")).hexdigest()[:10]
    return f"{safe}__{h}"

_BAD_FS_CHARS = re.compile(r'[<>:"/\\|?*\x00-\x1F]')  # Windows-illegal + control chars

def _safe_filename(s: str, max_len: int = 160) -> str:
    """
    Make a string safe to use as a filename on Windows/macOS/Linux.
    - Replaces illegal characters (including / and :) with '_'
    - Collapses whitespace
    - Truncates to avoid path-too-long issues
    """
    s = str(s)
    s = _BAD_FS_CHARS.sub("_", s)
    s = re.sub(r"\s+", " ", s).strip()
    if len(s) > max_len:
        s = s[:max_len].rstrip("_ ").strip()
    return s or "empty"


def export_inspection_packets(
    df: pd.DataFrame,
    out_dir: str | Path = "outputs",
    *,
    # ---- columns (edit if yours differ)
    record_id_col: str = "record_id",
    group_id_col: str = "dup_group_id",
    hierarchy_col: str = "records_hierarchy",
    parent_id_col: str = "parent_record_id",
    title_col: str = "title",
    authors_col: str = "authors_flat",
    server_col: str = "server_name",
    date_col: str | None = "date_first_seen",   # set None if you don't have it
    title_clean_col: str | None = None,         # e.g. "title_clean_v2" if you saved it
    # ---- packets
    n_random: int = 10,
    n_largest: int = 10,
    n_cross_server: int = 10,
    big_cluster_threshold: int = 50,            # red flag threshold
    random_seed: int = 13,
) -> dict:
    """
    Export dedupe clusters for manual inspection and an audit report.

    Outputs:
      outputs/
        dedupe_clusters_long.csv
        dedupe_clusters_summary.csv
        dedupe_audit_report.json
        dedupe_report.html
        packets/
          10_random_clusters/*.csv
          10_largest_clusters/*.csv
          10_cross_server_clusters/*.csv
          red_flags/*.csv
    Returns:
      report dict (also saved to JSON).
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    packets_dir = out_dir / "packets"
    packets_dir.mkdir(exist_ok=True)

    # -----------------------------
    # helpers
    # -----------------------------
    def _safe_cols(cols: list[str]) -> list[str]:
        return [c for c in cols if c in df.columns]

    def _count_duplicates(s: pd.Series) -> int:
        ss = s.astype("string").fillna("")
        return int(ss.str.startswith("parent - duplicate").sum())

    def _is_nonempty_string_series(s: pd.Series) -> pd.Series:
        ss = s.astype("string")
        return ss.notna() & (ss.str.len() > 0)

    # -----------------------------
    # clusterable rows
    # -----------------------------
    if group_id_col not in df.columns:
        raise ValueError(f"Missing required column: {group_id_col}")

    clustered_mask = _is_nonempty_string_series(df[group_id_col])
    clustered = df.loc[clustered_mask].copy()

    # -----------------------------
    # clusters_long (one row per record)
    # -----------------------------
    base_cols = [
        group_id_col,
        record_id_col,
        parent_id_col,
        hierarchy_col,
        server_col,
        date_col if date_col else "",
        title_col,
        title_clean_col if title_clean_col else "",
        authors_col,
    ]
    cols = _safe_cols([c for c in base_cols if c])

    clusters_long = clustered.loc[:, cols].copy()

    # cluster size per row
    clusters_long["cluster_size"] = clusters_long.groupby(group_id_col)[record_id_col].transform("size")

    # sort
    sort_cols = [group_id_col, "cluster_size"]
    asc = [True, False]
    if date_col and date_col in clusters_long.columns:
        clusters_long[date_col] = pd.to_datetime(clusters_long[date_col], errors="coerce")
        sort_cols.append(date_col)
        asc.append(True)

    clusters_long = clusters_long.sort_values(sort_cols, ascending=asc)
    clusters_long_path = out_dir / "dedupe_clusters_long.csv"
    clusters_long.to_csv(clusters_long_path, index=False)    
    clusters_long.to_pickle(out_dir / "dedupe_clusters_long.pkl")

    # -----------------------------
    # clusters_summary (one row per cluster)
    # -----------------------------
    def _join_uniq(x: pd.Series) -> str:
        vals = [str(v) for v in x.dropna().tolist()]
        return ";".join(sorted(set(vals)))

    agg = {record_id_col: "count"}
    if server_col in clusters_long.columns:
        agg[server_col] = _join_uniq
    if title_col in clusters_long.columns:
        agg[title_col] = "first"
    if date_col and date_col in clusters_long.columns:
        agg[date_col] = "min"

    cluster_summary = (
        clusters_long.groupby(group_id_col, dropna=False)
        .agg(agg)
        .rename(columns={
            record_id_col: "n_records",
            server_col: "servers",
            title_col: "example_title",
            date_col: "min_date",
        })
        .reset_index()
        .sort_values("n_records", ascending=False)
    )

    summary_path = out_dir / "dedupe_clusters_summary.csv"
    cluster_summary.to_csv(summary_path, index=False)

    # -----------------------------
    # audit report (JSON)
    # -----------------------------
    report = {}
    report["n_rows_total"] = int(len(df))
    report["n_rows_clustered"] = int(len(clustered))

    if hierarchy_col in df.columns:
        h = df[hierarchy_col].astype("string")
        report["n_parents"] = int(h.str.lower().eq("parent").sum())
        report["n_duplicates"] = _count_duplicates(df[hierarchy_col])
        report["n_unlabeled"] = int(df[hierarchy_col].isna().sum())
    else:
        report["n_parents"] = report["n_duplicates"] = report["n_unlabeled"] = 0

    report["n_clusters"] = int(cluster_summary[group_id_col].nunique()) if len(cluster_summary) else 0

    if len(cluster_summary):
        sizes = cluster_summary["n_records"]
        report["cluster_size_min"] = int(sizes.min())
        report["cluster_size_median"] = float(sizes.median())
        report["cluster_size_p95"] = float(sizes.quantile(0.95))
        report["cluster_size_max"] = int(sizes.max())
    else:
        report["cluster_size_min"] = report["cluster_size_median"] = report["cluster_size_p95"] = report["cluster_size_max"] = 0

    # cross-server clusters
    if "servers" in cluster_summary.columns:
        # servers field is "a;b;c" - count distinct servers
        n_srv = cluster_summary["servers"].fillna("").apply(lambda x: len([z for z in str(x).split(";") if z]))
        report["clusters_cross_server"] = int((n_srv >= 2).sum())
        report["clusters_single_server"] = int((n_srv == 1).sum())
    else:
        report["clusters_cross_server"] = report["clusters_single_server"] = 0

    # parent integrity checks
    if parent_id_col in df.columns and hierarchy_col in df.columns:
        dup_mask = df[hierarchy_col].astype("string").fillna("").str.startswith("parent - duplicate")
        report["dups_missing_parent_id"] = int(df.loc[dup_mask, parent_id_col].isna().sum())

        parent_mask = df[hierarchy_col].astype("string").fillna("").str.lower().eq("parent")
        report["parents_with_parent_id_filled"] = int(df.loc[parent_mask, parent_id_col].notna().sum())
    else:
        report["dups_missing_parent_id"] = 0
        report["parents_with_parent_id_filled"] = 0

    audit_json_path = out_dir / "dedupe_audit_report.json"
    with open(audit_json_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)

    # -----------------------------
    # red flags exports
    # -----------------------------
    red_flags_dir = packets_dir / "red_flags"
    red_flags_dir.mkdir(exist_ok=True)

    # 1) duplicates missing parent id
    if parent_id_col in df.columns and hierarchy_col in df.columns:
        dup_mask = df[hierarchy_col].astype("string").fillna("").str.startswith("parent - duplicate")
        bad = df.loc[dup_mask & df[parent_id_col].isna(), _safe_cols([record_id_col, group_id_col, hierarchy_col, server_col, title_col])].copy()
        bad.to_csv(red_flags_dir / "dups_missing_parent_id.csv", index=False)

    # 2) big clusters
    if len(cluster_summary):
        big_ids = cluster_summary.loc[cluster_summary["n_records"] >= big_cluster_threshold, group_id_col].tolist()
        if big_ids:
            big = clusters_long.loc[clusters_long[group_id_col].isin(big_ids), _safe_cols([group_id_col, record_id_col, parent_id_col, hierarchy_col, server_col, title_col])].copy()
            big.to_csv(red_flags_dir / f"big_clusters_{big_cluster_threshold}plus.csv", index=False)

    # -----------------------------
    # packet exporters
    # -----------------------------
    # def _export_cluster_csvs(cluster_ids: list[str], folder: Path):
    #     folder.mkdir(exist_ok=True)
    #     for gid in cluster_ids:
    #         part = clusters_long[clusters_long[group_id_col] == gid]
    #         part.to_csv(folder / f"cluster_{gid}.csv", index=False)

    def _export_cluster_csvs(cluster_ids, folder):
        folder = Path(folder)
        folder.mkdir(parents=True, exist_ok=True)  # <--- IMPORTANT
    
        for gid in cluster_ids:
            part = clusters_long[clusters_long[group_id_col].astype(str) == str(gid)]
            safe_gid = _safe_filename(gid)
            part.to_csv(folder / f"cluster_{safe_gid}.csv", index=False)
            # part.to_csv(folder / f"cluster_{_gid_slug(safe_gid)}.csv", index=False)

    
    # random clusters (from all clusters)
    random.seed(random_seed)
    all_ids = cluster_summary[group_id_col].astype(str).tolist()
    random_ids = random.sample(all_ids, k=min(n_random, len(all_ids))) if all_ids else []
    _export_cluster_csvs(random_ids, packets_dir / f"{len(random_ids)}_random_clusters")

    # largest clusters
    largest_ids = cluster_summary.head(n_largest)[group_id_col].astype(str).tolist()
    _export_cluster_csvs(largest_ids, packets_dir / f"{len(largest_ids)}_largest_clusters")

    # cross-server clusters
    cross_ids = []
    if "servers" in cluster_summary.columns:
        n_srv = cluster_summary["servers"].fillna("").apply(lambda x: len([z for z in str(x).split(";") if z]))
        cross_df = cluster_summary.loc[n_srv >= 2].copy()
        cross_ids = cross_df.head(n_cross_server)[group_id_col].astype(str).tolist()
    _export_cluster_csvs(cross_ids, packets_dir / f"{len(cross_ids)}_cross_server_clusters")

    # -----------------------------
    # simple HTML report
    # -----------------------------
    html_path = out_dir / "dedupe_report.html"

    def _escape(x) -> str:
        return ("" if pd.isna(x) else str(x)).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")

    top_preview = cluster_summary.head(50).copy()
    # make HTML table
    rows = []
    for _, r in top_preview.iterrows():
        gid = _escape(r[group_id_col])
        nrec = _escape(r.get("n_records", ""))
        srv = _escape(r.get("servers", ""))
        title = _escape(r.get("example_title", ""))
        rows.append(f"<tr><td>{gid}</td><td>{nrec}</td><td>{srv}</td><td>{title}</td></tr>")

    html = f"""
<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<title>Dedupe Audit Report</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 20px; }}
code {{ background: #f4f4f4; padding: 2px 4px; border-radius: 4px; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border: 1px solid #ddd; padding: 8px; vertical-align: top; }}
th {{ background: #f7f7f7; }}
.small {{ color: #555; font-size: 12px; }}
</style>
</head>
<body>
<h2>Dedupe Audit Report</h2>

<h3>Summary</h3>
<ul>
  <li>Total rows: <b>{report['n_rows_total']}</b></li>
  <li>Clustered rows: <b>{report['n_rows_clustered']}</b></li>
  <li>Clusters: <b>{report['n_clusters']}</b></li>
  <li>Parents: <b>{report['n_parents']}</b> | Duplicates: <b>{report['n_duplicates']}</b> | Unlabeled: <b>{report['n_unlabeled']}</b></li>
  <li>Cluster size (min/median/p95/max): <b>{report['cluster_size_min']}</b> / <b>{report['cluster_size_median']:.1f}</b> / <b>{report['cluster_size_p95']:.1f}</b> / <b>{report['cluster_size_max']}</b></li>
  <li>Cross-server clusters: <b>{report['clusters_cross_server']}</b> | Single-server clusters: <b>{report['clusters_single_server']}</b></li>
  <li>Duplicates missing parent_id: <b>{report['dups_missing_parent_id']}</b></li>
</ul>

<p class="small">
Files saved in <code>{out_dir.as_posix()}</code>:
<br/>- dedupe_clusters_long.csv
<br/>- dedupe_clusters_summary.csv
<br/>- dedupe_audit_report.json
<br/>- packets/...
</p>

<h3>Top 50 largest clusters (preview)</h3>
<table>
  <thead><tr><th>dup_group_id</th><th>n_records</th><th>servers</th><th>example_title</th></tr></thead>
  <tbody>
    {''.join(rows)}
  </tbody>
</table>

</body>
</html>
""".strip()

    html_path.write_text(html, encoding="utf-8")

    # return
    report["files_written"] = {
        "clusters_long": str(clusters_long_path),
        "clusters_summary": str(summary_path),
        "audit_json": str(audit_json_path),
        "html_report": str(html_path),
        "packets_dir": str(packets_dir),
    }
    return report


In [19]:
report = export_inspection_packets(
    data_group_merge,
    hierarchy_col="records_hierarchy",
    out_dir="outputs_new",
    date_col="date_first_seen",   # or None
    title_clean_col="title_clean_v2",  # optional
    n_random=10,
    n_largest=10,
    n_cross_server=10,
    big_cluster_threshold=50,
)
report


{'n_rows_total': 8084860,
 'n_rows_clustered': 1113404,
 'n_parents': 7467120,
 'n_duplicates': 617740,
 'n_unlabeled': 0,
 'n_clusters': 495664,
 'cluster_size_min': 1,
 'cluster_size_median': 2.0,
 'cluster_size_p95': 3.0,
 'cluster_size_max': 702,
 'clusters_cross_server': 194959,
 'clusters_single_server': 300705,
 'dups_missing_parent_id': 0,
 'parents_with_parent_id_filled': 0,
 'files_written': {'clusters_long': 'outputs_new/dedupe_clusters_long.csv',
  'clusters_summary': 'outputs_new/dedupe_clusters_summary.csv',
  'audit_json': 'outputs_new/dedupe_audit_report.json',
  'html_report': 'outputs_new/dedupe_report.html',
  'packets_dir': 'outputs_new/packets'}}

In [20]:
from pathlib import Path
OUT_DIR = Path("outputs_new"); OUT_DIR.mkdir(exist_ok=True)

cols_new = [
    "record_id",
    'server_name',
    "records_hierarchy_copy3",
    "records_hierarchy_copy2_copy",
    "records_hierarchy_copy2",
    "records_hierarchy_copy",
    "records_hierarchy",
    "parent_record_id",
    "dup_group_id",
    "authors_fp_tokenbag",
    "authors_fp_last_initial",
    "authors_fp_last",
    "title_clean_v2",
]
cols_new = [c for c in cols_new if c in data_group_merge.columns]

data_group_merge[cols_new].to_parquet(OUT_DIR / "dedupe_new_columns.parquet", index=False)
print("Saved:", OUT_DIR / "dedupe_new_columns.parquet", "| cols:", cols_new)


Saved: outputs_new/dedupe_new_columns.parquet | cols: ['record_id', 'server_name', 'records_hierarchy', 'parent_record_id', 'dup_group_id', 'authors_fp_tokenbag', 'authors_fp_last_initial', 'authors_fp_last', 'title_clean_v2']


# explorations

In [21]:
data_group_merge["records_hierarchy"].value_counts(dropna=False)#.head(60)

records_hierarchy
parent                                            7467120
parent - duplicate (SSRN)                          135606
parent - duplicate (Zenodo)                         98767
parent - duplicate (arXiv)                          85008
parent - duplicate (Research Square)                59931
                                                   ...   
parent - duplicate (SAE Mobilus®)                       1
parent - duplicate (Therapoid)                          1
parent - duplicate (WikiJournal of Humanities)          1
parent - duplicate (WikiJournal of Science)             1
parent - duplicate (searchRxiv)                         1
Name: count, Length: 109, dtype: int64

In [22]:
data["records_hierarchy"].value_counts(dropna=False)#.head(60)

records_hierarchy
parent                                            7467120
parent - duplicate (SSRN)                          135606
parent - duplicate (Zenodo)                         98767
parent - duplicate (arXiv)                          85008
parent - duplicate (Research Square)                59931
                                                   ...   
parent - duplicate (SAE Mobilus®)                       1
parent - duplicate (Therapoid)                          1
parent - duplicate (WikiJournal of Humanities)          1
parent - duplicate (WikiJournal of Science)             1
parent - duplicate (searchRxiv)                         1
Name: count, Length: 109, dtype: int64

In [23]:
data_group_merge

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084855,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None,parent,None,None
8084856,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None,parent,None,None
8084857,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None,parent,None,None
8084858,openalex::W999790414,viXra,open

In [24]:
print(data_group_merge["records_hierarchy"].value_counts(dropna=False).head(60))

records_hierarchy
parent                                                                                       7467120
parent - duplicate (SSRN)                                                                     135606
parent - duplicate (Zenodo)                                                                    98767
parent - duplicate (arXiv)                                                                     85008
parent - duplicate (Research Square)                                                           59931
parent - duplicate (AgEcon Search)                                                             34887
parent - duplicate (Open Science Framework)                                                    21864
parent - duplicate (HAL)                                                                       21136
parent - duplicate (eLife)                                                                     20267
parent - duplicate (RePEc: Research Papers in Economics)                 

In [25]:
print(data_group_merge["parent_record_id"].value_counts(dropna=False).head(60))

parent_record_id
None                                          6973788
<NA>                                           493332
datacite::10.5281/zenodo.14289192                 701
datacite::10.5281/zenodo.15609432                 568
datacite::10.22004/ag.econ.138684                 449
datacite::10.22004/ag.econ.136112                 389
datacite::10.22004/ag.econ.286599                 300
datacite::10.5281/zenodo.15832876                 285
datacite::10.5281/zenodo.15161515                 260
crossref::10.31219/osf.io/ed2a9                   209
crossref::10.32388/174914                         183
datacite::10.5281/zenodo.6784019                  181
datacite::10.5281/zenodo.14599183                 176
datacite::10.5281/zenodo.15631517                 166
crossref::10.2139/ssrn.2721313                    163
datacite::10.5281/zenodo.15408957                 161
datacite::10.5281/zenodo.15690627                 158
crossref::10.31237/osf.io/7j3am                   149
crossref::1

In [26]:
pattern = "fuzzy::las"


mask = data_group_merge['dup_group_id'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

0


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup


In [27]:
# result['relations_json'][281751]

In [28]:
data_group_merge[data_group_merge["record_id"]=='crossref::10.26434/chemrxiv-2021-cj17s']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
281688,crossref::10.26434/chemrxiv-2021-cj17s,ChemRxiv,crossref,10.26434/chemrxiv-2021-cj17s,https://doi.org/10.26434/chemrxiv-2021-cj17s,https://chemrxiv.org/engage/chemrxiv/article-d...,First-Principles Plane-Wave-Based Exploration ...,"Ertural, Christina; Stoffel, Ralf; Müller, Pet...",RWTH Aachen University,None,None,None,,,,,false,None,None,None,None,parent,2021-07-26,2021,parent,<NA>,crossref::10.26434/chemrxiv-2021-cj17s,None,first principles plane wave based exploration ...,dronskowski|r;ertural|c;muller|p;stoffel|r;vogt|c,None,parent,None,None,parent,None,None


In [29]:
data_group_merge[data_group_merge["parent_record_id"]=='crossref::10.26434/chemrxiv-2022-fd190']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
282703,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,relaxed::A1_tokenbag_exact::8264,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282758,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,8105542989974901890,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282765,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,8105542989974901890,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
283682,crossref::10.26434/chemrxiv-2022-fd190-v5,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v5,https://doi.org/10.26434/chemrxiv-2022-fd190-v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv

In [30]:
data_group_merge[data_group_merge["dup_group_id"]=='crossref::10.26434/chemrxiv-2022-fd190']

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
282432,crossref::10.26434/chemrxiv-2022-fd190,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190,https://doi.org/10.26434/chemrxiv-2022-fd190,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,None,None,,,,,false,None,None,None,None,parent,2022-03-18,2022,parent,<NA>,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent,None,relaxed::A1_tokenbag_exact::8264,parent,None,crossref::10.26434/chemrxiv-2022-fd190
282703,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,relaxed::A1_tokenbag_exact::8264,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282758,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,8105542989974901890,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282765,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-20

In [31]:
data_group_merge[data_group_merge["dup_group_id"]=='crossref::10.26434/chemrxiv-2022-fd190']['title'].value_counts()

title
Selectivity and Intrinsic Activity of Functionalized Cu Surfaces: Can the CO2 Reduction Reaction be Improved on Cu?    5
Can the CO2 Reduction Reaction be Improved on Cu: Selectivity and Intrinsic Activity of Functionalized Cu Surfaces     1
Name: count, dtype: int64

In [32]:
pattern = "chemrxiv.11846943"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

9


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
276876,crossref::10.26434/chemrxiv.11846943.v1,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v1,https://doi.org/10.26434/chemrxiv.11846943.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Bhanu, Divya; Alluri, Anajani",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-02-13,2020,parent,<NA>,crossref::10.26434/chemrxiv.11846943.v1,alluri_anajani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None,parent - duplicate (Open Science Framework),crossref::10.31219/osf.io/2zuea,16256016634036771957,parent,None,crossref::10.26434/chemrxiv.11846943.v1
276912,crossref::10.26434/chemrxiv.11846943.v2,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v2,https://doi.org/10.26434/chemrxiv.11846943.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Bhanu, Divya; Alluri, Anajani",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-02-14,2020,parent,crossref::10.26434/chemrxiv.11846943.v1,crossref::10.26434/chemrxiv.11846943.v1,alluri_anajani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None,parent - duplicate (Open Science Framework),crossref::10.31219/osf.io/2zuea,fuzzy::tokenbag::alluri_anajani;arun_shanker;b...,parent - duplicate (version-link),crossref::10.26434/chemrxiv.11846943.v1,crossref::10.26434/chemrxiv.11846943.v1
276946,crossref::10.26434/chemrxiv.11846943.v3,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v3,https://doi.org/10.26434/chemrxiv.11846943.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Analysis of Whole Genome Sequences and Homolog...,"Shanker, Arun; Alluri, Anjani; Bhanu, Divya",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-02-17,2020,parent,crossref::10.26434/chemrxiv.11846943.v1,crossref::10.26434/chemrxiv.11846943.v1,alluri_anjani;arun_shanker;bhanu_divya,analysis of whole genome sequences and homolog...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.11846943.v2,relaxed::A1_tokenbag_exact::8040,parent - duplicate (version-link),crossref::10.26434/chemrxiv.11846943.v1,crossref::10.26434/chemrxiv.11846943.v1
276975,crossref::10.26434/chemrxiv.11846943.v4,ChemRxiv,crossref,10.26434/chemrxiv.11846943.v4,https://doi.org/10.26434/chemrxiv.11846943.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Whole Genome Sequences Analysis and Homology M...,"Shanker, Arun; Alluri, Anjani; Bhanu, Divya",Central Research Institute for Dryland Agricul...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.11846943.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-02-21,2020,parent,crossref::10.26434/chemrxiv.11846943.v1,crossref::10.26434/chemrxiv.1

In [33]:
result['records_hierarchy'].value_counts()

records_hierarchy
parent - duplicate (ChemRxiv)    8
parent                           1
Name: count, dtype: int64

In [34]:
result['title'].value_counts()

title
Whole Genome Sequence Analysis and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein 3 of the SARS-CoV-2 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties                       5
Analysis of Whole Genome Sequences and Homology Modelling of a 3-C Like Peptidase and a Non-Structural Protein of the Novel Coronavirus COVID-19 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties    2
Analysis of Whole Genome Sequences and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein of the Novel Coronavirus COVID-19 Shows Protein Ligand Interaction with an Aza-Peptide and a Noncovalent Lead Inhibitor with Possible Antiviral Properties     1
Whole Genome Sequences Analysis and Homology Modelling of a 3C Like Peptidase and a Non-Structural Protein 3 of the SARS-CoV-2 Shows Protein Ligand Interaction with an Aza-Peptide a

In [35]:
result['authors_flat'].value_counts()

authors_flat
Shanker, Arun; Alluri, Anjani; Bhanu, Divya                      3
Shanker, Arun; Bhanu, Divya; Alluri, Anjani                      3
Shanker, Arun; Bhanu, Divya; Alluri, Anajani                     2
Shanker, Arun; Bhanu, Divya; Alluri, Anjani; Gupta, Samriddhi    1
Name: count, dtype: int64

In [36]:
pattern = "10.26434/chemrxiv-2022-fd190"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

6


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
282432,crossref::10.26434/chemrxiv-2022-fd190,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190,https://doi.org/10.26434/chemrxiv-2022-fd190,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,None,None,,,,,false,None,None,None,None,parent,2022-03-18,2022,parent,<NA>,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent,None,relaxed::A1_tokenbag_exact::8264,parent,None,crossref::10.26434/chemrxiv-2022-fd190
282703,crossref::10.26434/chemrxiv-2022-fd190-v2,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v2,https://doi.org/10.26434/chemrxiv-2022-fd190-v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-04-25,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;alonso_hernandez_rosas;christ...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,relaxed::A1_tokenbag_exact::8264,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282758,crossref::10.26434/chemrxiv-2022-fd190-v3,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v3,https://doi.org/10.26434/chemrxiv-2022-fd190-v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-02,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-2022-fd190,8105542989974901890,parent - duplicate (version-link),crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190
282765,crossref::10.26434/chemrxiv-2022-fd190-v4,ChemRxiv,crossref,10.26434/chemrxiv-2022-fd190-v4,https://doi.org/10.26434/chemrxiv-2022-fd190-v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Selectivity and Intrinsic Activity of Function...,"Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong...",University of Copenhagen; Aarhus University,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv-2022-fd190;10.26434/chemrxiv...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2022-05-04,2022,parent,crossref::10.26434/chemrxiv-2022-fd190,crossref::10.26434/chemrxiv-2022-fd190,alexander_bagger;christensen_oliver;daasbjerg_...,selectivity and intrinsic activity of function...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv-20

In [37]:
result['title'].value_counts()

title
Selectivity and Intrinsic Activity of Functionalized Cu Surfaces: Can the CO2 Reduction Reaction be Improved on Cu?    5
Can the CO2 Reduction Reaction be Improved on Cu: Selectivity and Intrinsic Activity of Functionalized Cu Surfaces     1
Name: count, dtype: int64

In [38]:
result['authors_flat'].value_counts()

authors_flat
Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong; Bagger, Alexander; Vang Lauritsen, Jeppe; Uttrup Pedersen, Steen; Daasbjerg, Kim; Rossmeisl, Jan                             5
Christensen, Oliver; Zhao, Siqi; Sun, Zhaozong; Bagger, Alexander; Rosas-Hernández, Alonso; Vang Lauritsen, Jeppe; Uttrup Pedersen, Steen; Daasbjerg, Kim; Rossmeisl, Jan    1
Name: count, dtype: int64

In [39]:
sample_titles = data_group_merge.sample(5)[['title', 'authors_flat']].title
data_group_merge[data_group_merge.title.isin(sample_titles)][['title','authors_flat','records_hierarchy','date_first_seen']]

,title,authors_flat,records_hierarchy,date_first_seen
151829,A Three-Legged Approach Towards Improved Devel...,"Hamilton, Jane; Quirk, Michael; Dyer, Rodd; Sc...",parent,2019-10-17
4540961,Obat Aborsi Semarang ( Ampuh ) 082314597755 Kl...,leopribady,parent,2020-05-24
4541104,Obat Aborsi Semarang ( Ampuh ) 082314597755 Kl...,hellobidan,parent - duplicate (Thesis Commons),2020-11-15
5974322,$η'$ Production in Nucleus-Nucleus collisions ...,"Torrieri, Giorgio",parent,2022-03-03
6073767,Poisson-Boltzmann study of the effective elect...,"Majee, Arghya; Bier, Markus; Dietrich, S.",parent,2022-03-05
7562362,Non-intrusive reduced order models for partiti...,"Tiba, Azzeddine; Dairay, Thibault; de Vuyst, F...",parent,2023-06-14


In [40]:
pattern = "10.26434/chemrxiv.13102877"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

32


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
279824,crossref::10.26434/chemrxiv.13102877.v1,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v1,https://doi.org/10.26434/chemrxiv.13102877.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-10-23,2020,parent,<NA>,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,datta|s,None,parent,None,16854196709595633026,parent,None,crossref::10.26434/chemrxiv.13102877.v1
279854,crossref::10.26434/chemrxiv.13102877.v2,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v2,https://doi.org/10.26434/chemrxiv.13102877.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-10-27,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v1,16854196709595633026,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279881,crossref::10.26434/chemrxiv.13102877.v3,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v3,https://doi.org/10.26434/chemrxiv.13102877.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-06,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,datta|s,None,parent,None,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279912,crossref::10.26434/chemrxiv.13102877.v4,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v4,https://doi.org/10.26434/chemrxiv.13102877.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-09,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v3,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279952,crossref::10.26434/chemrxiv.13102877.v5,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v5,https://doi.org/10.26434/chemrxiv.13102877.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Sho

In [41]:
result['title'].value_counts()

title
Aptamers for Detection and Diagnostics (ADD) is a proposed mobile app acquiring optical data from conjugated quantum nanodots to identify molecules indicating presence of SARS-CoV-2 virus: Why public health and healthcare need smartphone sensors as a platform for early detection and prevention    28
Aptamers for Detection and Diagnostics (ADD): Can mobile systems process optical data from aptamer sensors to identify molecules indicating presence of SARS-CoV-2 virus? Should healthcare explore aptamers as drugs for prevention as well as its use as adjuvants with antibodies and vaccines?         2
Aptamers for Detection and Diagnostics (ADD): Proposed Mobile App Acquiring Optical Data from Aptamers Conjugated with Quantum Nanodots May Detect Harmful Molecules as well as SARS-CoV-2                                                                                                                 1
Aptamers for Detection and Diagnostics (ADD): Proposed Mobile App Acquiring Optical Data fr

In [42]:
result['authors_flat'].value_counts()

authors_flat
Datta, Shoumen    32
Name: count, dtype: int64

In [43]:
result['dup_group_id'].value_counts()

dup_group_id
crossref::10.26434/chemrxiv.13102877.v1    32
Name: count, dtype: int64

In [44]:
result['parent_record_id'].value_counts()

parent_record_id
crossref::10.26434/chemrxiv.13102877.v1    31
Name: count, dtype: int64

In [45]:
result['records_hierarchy'].value_counts()

records_hierarchy
parent - duplicate (ChemRxiv)    31
parent                            1
Name: count, dtype: int64

In [46]:
result['is_version_of'].value_counts()

is_version_of
10.26434/chemrxiv.13102877.v2;10.26434/chemrxiv.13102877.v3;10.26434/chemrxiv.13102877.v4;10.26434/chemrxiv.13102877.v5;10.26434/chemrxiv.13102877.v6;10.26434/chemrxiv.13102877.v7;10.26434/chemrxiv.13102877.v8;10.26434/chemrxiv.13102877.v9;10.26434/chemrxiv.13102877.v10;10.26434/chemrxiv.13102877.v11;10.26434/chemrxiv.13102877.v12;10.26434/chemrxiv.13102877.v13;10.26434/chemrxiv.13102877.v14;10.26434/chemrxiv.13102877.v15;10.26434/chemrxiv.13102877.v16;10.26434/chemrxiv.13102877.v17;10.26434/chemrxiv.13102877.v18;10.26434/chemrxiv.13102877.v19;10.26434/chemrxiv.13102877.v20;10.26434/chemrxiv.13102877.v21;10.26434/chemrxiv.13102877.v22;10.26434/chemrxiv.13102877.v23;10.26434/chemrxiv.13102877.v24;10.26434/chemrxiv.13102877.v25;10.26434/chemrxiv.13102877.v26;10.26434/chemrxiv.13102877.v27;10.26434/chemrxiv.13102877.v28;10.26434/chemrxiv.13102877.v29;10.26434/chemrxiv.13102877.v30;10.26434/chemrxiv.13102877.v31;10.26434/chemrxiv.13102877.v32;10.33774/chemrxiv-2021-l393t-v33;

In [47]:
pattern = "10.26434/chemrxiv.131028"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

32


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
279824,crossref::10.26434/chemrxiv.13102877.v1,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v1,https://doi.org/10.26434/chemrxiv.13102877.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-10-23,2020,parent,<NA>,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,datta|s,None,parent,None,16854196709595633026,parent,None,crossref::10.26434/chemrxiv.13102877.v1
279854,crossref::10.26434/chemrxiv.13102877.v2,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v2,https://doi.org/10.26434/chemrxiv.13102877.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-10-27,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v1,16854196709595633026,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279881,crossref::10.26434/chemrxiv.13102877.v3,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v3,https://doi.org/10.26434/chemrxiv.13102877.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-06,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,datta|s,None,parent,None,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279912,crossref::10.26434/chemrxiv.13102877.v4,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v4,https://doi.org/10.26434/chemrxiv.13102877.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-09,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v3,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279952,crossref::10.26434/chemrxiv.13102877.v5,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v5,https://doi.org/10.26434/chemrxiv.13102877.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Sho

In [48]:
pattern = "Exploring the Relationship of Enrollmen"


mask = data_group_merge['title'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

3


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
190,crossref::10.33774/apsa-2019-nd8n2,APSA Preprints,crossref,10.33774/apsa-2019-nd8n2,https://doi.org/10.33774/apsa-2019-nd8n2,https://preprints.apsanet.org/engage/apsa/arti...,Exploring the Relationship of Enrollment in In...,"Collier, Daniel A.; Fitzpatrick, Dan; Marsican...",W.E. Upjohn Institute for Employment Research;...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.33774/apsa-2019-nd8n2-v2,,,,false,None,None,None,None,parent,2019-10-04,2019,version,<NA>,crossref::10.33774/apsa-2019-nd8n2,a_collier_daniel;christopher_marsicano_r;dan_f...,exploring the relationship of enrollment in in...,collier|d;fitzpatrick|d;marsicano|c,None,parent,None,10655472821248023217,parent,None,crossref::10.33774/apsa-2019-nd8n2
215,crossref::10.33774/apsa-2019-nd8n2-v2,APSA Preprints,crossref,10.33774/apsa-2019-nd8n2-v2,https://doi.org/10.33774/apsa-2019-nd8n2-v2,https://preprints.apsanet.org/engage/apsa/arti...,Exploring the Relationship of Enrollment in In...,"Collier, Daniel A.; Fitzpatrick, Dan; Marsican...",W.E. Upjohn Institute for Employment Research;...,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.33774/apsa-2019-nd8n2,,,,false,None,None,None,None,parent - duplicate (APSA Preprints),2020-04-06,2020,version,crossref::10.33774/apsa-2019-nd8n2,crossref::10.33774/apsa-2019-nd8n2,a_collier_daniel;christopher_marsicano_r;dan_f...,exploring the relationship of enrollment in in...,None,None,parent - duplicate (APSA Preprints),crossref::10.33774/apsa-2019-nd8n2,10655472821248023217,parent - duplicate (version-link),crossref::10.33774/apsa-2019-nd8n2,crossref::10.33774/apsa-2019-nd8n2
3710568,crossref::10.2139/ssrn.3757559,SSRN,crossref,10.2139/ssrn.3757559,https://doi.org/10.2139/ssrn.3757559,https://www.ssrn.com/abstract=3757559,Exploring the Relationship of Enrollment in ID...,"Collier, Daniel; Fitzpatrick, Dan; Marsicano, ...",None,None,None,None,,,,,false,None,None,None,None,parent - duplicate (SSRN),2021-01-08,2021,parent,crossref::10.33774/apsa-2019-nd8n2,crossref::10.33774/apsa-2019-nd8n2,christopher_marsicano;collier_daniel;dan_fitzp...,exploring the relationship of enrollment in id...,collier|d;fitzpatrick|d;marsicano|c,None,parent - duplicate (APSA Preprints),crossref::10.33774/apsa-2019-nd8n2,fuzzy::last_initial::collier|d;fitzpatrick|d;m...,parent - duplicate (APSA Preprints),crossref::10.33774/apsa-2019-nd8n2,fuzzy::last_initial::collier|d;fitzpatrick|d;m...


In [49]:
result[['record_id','server_name','records_hierarchy','parent_record_id','dup_group_id','date_first_seen',]]

,record_id,server_name,records_hierarchy,parent_record_id,dup_group_id,date_first_seen
190,crossref::10.33774/apsa-2019-nd8n2,APSA Preprints,parent,<NA>,crossref::10.33774/apsa-2019-nd8n2,2019-10-04
215,crossref::10.33774/apsa-2019-nd8n2-v2,APSA Preprints,parent - duplicate (APSA Preprints),crossref::10.33774/apsa-2019-nd8n2,crossref::10.33774/apsa-2019-nd8n2,2020-04-06
3710568,crossref::10.2139/ssrn.3757559,SSRN,parent - duplicate (SSRN),crossref::10.33774/apsa-2019-nd8n2,crossref::10.33774/apsa-2019-nd8n2,2021-01-08


In [50]:
pattern = "10.26434/chemrxiv.13102877"


mask = data_group_merge['doi'].str.contains(pattern, regex=False, na=False)
result = data_group_merge[mask]
print(len(result))
result

32


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup,records_hierarchy_unify_version_backup,parent_record_id_unify_version_backup,dup_group_id_unify_version_backup
279824,crossref::10.26434/chemrxiv.13102877.v1,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v1,https://doi.org/10.26434/chemrxiv.13102877.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-10-23,2020,parent,<NA>,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,datta|s,None,parent,None,16854196709595633026,parent,None,crossref::10.26434/chemrxiv.13102877.v1
279854,crossref::10.26434/chemrxiv.13102877.v2,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v2,https://doi.org/10.26434/chemrxiv.13102877.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-10-27,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v1,16854196709595633026,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279881,crossref::10.26434/chemrxiv.13102877.v3,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v3,https://doi.org/10.26434/chemrxiv.13102877.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-06,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,datta|s,None,parent,None,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279912,crossref::10.26434/chemrxiv.13102877.v4,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v4,https://doi.org/10.26434/chemrxiv.13102877.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (ChemRxiv),2020-11-09,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v3,15378799223749550050,parent - duplicate (version-link),crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1
279952,crossref::10.26434/chemrxiv.13102877.v5,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v5,https://doi.org/10.26434/chemrxiv.13102877.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Sho